#### AUTRE FICHIER

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, classification_report
import numpy as np

Type de 'date_achat' : datetime64[ns]


In [16]:


# 1. Chargement et nettoyage
file_path = "../code/achat_prod_fournisseur_stock.csv"
df = pd.read_csv(file_path, parse_dates=["date_achat"])
# Conversion en datetime
df['date_achat'] = pd.to_datetime(df['date_achat'], dayfirst=True, errors='coerce')

df.columns = df.columns.str.strip().str.lower()

# 2. Calcul des KPI principaux
# 2.1 HHI fournisseurs (part de quantité achetée)
part_fourn = df.groupby("id_fournisseur")["quantité"].sum() / df["quantité"].sum()
hhi_fournisseurs = (part_fourn**2).sum() * 10000

# 2.2 Délai moyen de livraison (jours)
delai_moyen = df["délai_livraison_jours"].mean()

# 2.3 Fiabilité moyenne fournisseurs (%)
fiabilite_moyenne = df["fiabilité"].mean()

# 2.4 Rotation des stocks (consommation annuelle / stock moyen)
# Estimation consommation annuelle par produit
consommation_annuelle = df.groupby("id_produit")["quantité"].sum()  # suppose 1 an de données
stock_moyen = df.groupby("id_produit")["niveau_stock"].mean().replace(0, np.nan)
rotation_par_produit = consommation_annuelle / stock_moyen
rotation_stock_moyenne = rotation_par_produit.mean()

# 2.5 Coût unitaire moyen (€)
cout_unitaire_moyen = (df["montant_total"].sum() / df["quantité"].sum())

kpi_summary = {
    "HHI fournisseurs": hhi_fournisseurs,
    "Délai moyen de livraison (jours)": delai_moyen,
    "Fiabilité moyenne (%)": fiabilite_moyenne,
    "Rotation stock moyenne": rotation_stock_moyenne,
    "Coût unitaire moyen (€)": cout_unitaire_moyen
}

# Affichage du résumé KPI
print("Résumé des KPI Supply Chain:")
for k, v in kpi_summary.items():
    print(f"- {k}: {v:.2f}")



Résumé des KPI Supply Chain:
- HHI fournisseurs: 1301.51
- Délai moyen de livraison (jours): 7.96
- Fiabilité moyenne (%): 0.70
- Rotation stock moyenne: 25.00
- Coût unitaire moyen (€): 156.07


C:\Users\Djo\AppData\Local\Temp\ipykernel_14496\2596741604.py:5: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['date_achat'] = pd.to_datetime(df['date_achat'], dayfirst=True, errors='coerce')


In [18]:
# 3. Feature Engineering pour ML
df["mois"] = df["date_achat"].dt.month
df["jour_semaine"] = df["date_achat"].dt.dayofweek
df["retard"] = (df["délai_livraison_jours"] > df["délai_moyen_jours"]).astype(int)

# Sélection de features communes
features = ["quantité", "prix_unitaire", "mois", "jour_semaine", "niveau_stock"]
df_ml = df.dropna(subset=features + ["délai_livraison_jours", "retard"])


In [20]:

# 4. Cas d'usage ML
# 4.1 Régression: prédiction du délai de livraison
X_reg = df_ml[features]
y_reg = df_ml["délai_livraison_jours"]
Xr_train, Xr_test, yr_train, yr_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
reg_model = LinearRegression().fit(Xr_train, yr_train)
yr_pred = reg_model.predict(Xr_test)
mse = mean_squared_error(yr_test, yr_pred)
rmse = np.sqrt(mse)
print("\nRégression - RMSE délai de livraison:", rmse)


Régression - RMSE délai de livraison: 3.739837728581802


In [21]:

# 4.2 Classification: probabilité de retard (> délai moyen)
X_clf = df_ml[features]
y_clf = df_ml["retard"]
Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)
clf_model = LogisticRegression(max_iter=1000).fit(Xc_train, yc_train)
yc_pred = clf_model.predict(Xc_test)
print("\nClassification - Rapport de performance:")
print(classification_report(yc_test, yc_pred))

# 4.3 Clustering: segmentation fournisseurs
fourn_features = df.groupby("id_fournisseur").agg({
    "quantité": "sum",
    "délai_livraison_jours": "mean",
    "fiabilité": "mean"
}).dropna()
kmeans = KMeans(n_clusters=3, random_state=42).fit(fourn_features)
fourn_features["cluster"] = kmeans.labels_
print("\nClustering fournisseurs - Répartition par cluster:")
print(fourn_features["cluster"].value_counts())

# 4.4 Forecasting (prévision simple) : agrégation mensuelle volumes
monthly_demand = df.groupby(["année", "mois", "id_produit"])["quantité"].sum().reset_index()
print("\nExtrait prévision demande (agrégations mensuelles):")
print(monthly_demand.head())




Classification - Rapport de performance:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       988
           1       0.51      1.00      0.67      1012

    accuracy                           0.51      2000
   macro avg       0.25      0.50      0.34      2000
weighted avg       0.26      0.51      0.34      2000



c:\Users\Djo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Djo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Djo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier


Clustering fournisseurs - Répartition par cluster:
cluster
0    42
2     5
1     3
Name: count, dtype: int64

Extrait prévision demande (agrégations mensuelles):
   année  mois id_produit  quantité
0   2024   1.0     P00001       200
1   2024   1.0     P00002       110
2   2024   1.0     P00003        50
3   2024   1.0     P00004       225
4   2024   1.0     P00005       160
